# Lab | SQL Queries 9

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals. You have been using this database for a couple labs already, but if you need to get the data again, refer to the official [installation link](https://dev.mysql.com/doc/sakila/en/sakila-installation.html).

The database is structured as follows:
![DB schema](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/database-sakila-schema.png)

### Instructions

In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.

- Create a table `rentals_may` to store the data from rental table with information for the month of May.
- Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
- Create a table `rentals_june` to store the data from rental table with information for the month of June.
- Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
- Check the number of rentals for each customer for May.
- Check the number of rentals for each customer for June.
- Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.

- Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

  **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).


In [87]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

In [88]:
#Create a Python connection with SQL database

#import getpass
#password = getpass.getpass()


connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)
may_rent = pd.read_sql_query('select customer_id, count(rental_id) as count_rentals from sakila.rentals_may group by customer_id', engine)
june_rent = pd.read_sql_query('select customer_id, count(rental_id) as count_rentals from sakila.rentals_june group by customer_id', engine)



In [89]:
# Display the two dfs

display(may_rent)
display(june_rent)

,customer_id,count_rentals
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5
...,...,...
515,191,2
516,351,1
517,10,1
518,136,1


,customer_id,count_rentals
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6
...,...,...
585,412,1
586,335,1
587,226,2
588,22,1


In [90]:
# Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

# Function for each customer
def monthly_activity(df1, df2):
    '''
        Function to compare rental activity between May and June for each customer

        Parameters:
        - df1: DataFrame containing rental data for May
        - df2: DataFrame containing rental data for June

        Steps:
        1. Merge the DataFrames on 'customer_id'.
        2. Rename columns for clarity.
        3. Drop rows with missing values.
        4. Set 'customer_id' as the index and sort the DataFrame.
        5. Iterate over rows and compare rental counts for May and June.
        6. Print the result for each customer: more rentals in May, more in June, or equal counts.

    '''
    # Merge the DataFrames on 'customer_id'
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer')
   
    # Rename columns for clarity
    merged_df = merged_df.rename(columns={
        'count_rentals_x': 'may_rental_count',
        'count_rentals_y': 'june_rental_count'
    })

    # Drop rows with missing values
    merged_df = merged_df.dropna()

    # Set 'customer_id' as the index and sort the DataFrame
    merged_df = merged_df.set_index('customer_id')
    merged_df = merged_df.sort_index()

    # Iterate over rows and compare rental counts for May and June
    for index, row in merged_df.iterrows():
        customer_id = index
        may_rentals = row['may_rental_count']
        june_rentals = row['june_rental_count']

        # Print the result for each customer
        if may_rentals > june_rentals:
            print(f"Customer {customer_id}: More rentals in May")
        elif may_rentals < june_rentals:
            print(f"Customer {customer_id}: More rentals in June")
        else:
            print(f"Customer {customer_id}: Equal rentals in May and June")

    
monthly_activity(may_rent, june_rent)  

Customer 1: More rentals in June
Customer 2: Equal rentals in May and June
Customer 3: More rentals in June
Customer 5: More rentals in June
Customer 6: More rentals in June
Customer 7: Equal rentals in May and June
Customer 8: More rentals in June
Customer 9: More rentals in May
Customer 10: More rentals in June
Customer 11: More rentals in June
Customer 12: More rentals in June
Customer 14: More rentals in May
Customer 16: Equal rentals in May and June
Customer 17: Equal rentals in May and June
Customer 18: More rentals in June
Customer 19: More rentals in May
Customer 20: More rentals in June
Customer 21: More rentals in June
Customer 22: More rentals in May
Customer 23: More rentals in June
Customer 24: More rentals in June
Customer 25: More rentals in June
Customer 26: More rentals in June
Customer 27: More rentals in June
Customer 28: More rentals in June
Customer 29: More rentals in June
Customer 32: More rentals in June
Customer 33: More rentals in June
Customer 35: More rental

In [91]:
# Function for a selected customer from the user
def input_monthly_activity(df1, df2):
    '''
        Function to input a customer number and compare rental activity between May and June for the specified customer.

        Parameters:
        - df1: DataFrame containing rental data for May
        - df2: DataFrame containing rental data for June

        Steps:
        1. Merge the DataFrames on 'customer_id'.
        2. Rename columns for clarity.
        3. Drop rows with missing values.
        4. Get user input for the customer number.
        5. Check if the input customer number is valid.
        6. Retrieve the row for the specified customer.
        7. Compare rental counts for May and June.
        8. Print the result for the chosen customer: more rentals in May, more in June, or equal counts.

    '''
    # Merge the DataFrames on 'customer_id'
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer')
    
    # Rename columns for clarity
    merged_df = merged_df.rename(columns={
        'count_rentals_x': 'may_rental_count',
        'count_rentals_y': 'june_rental_count'
    })

    # Drop rows with missing values
    merged_df = merged_df.dropna()

    # Get user input for the customer number
    customer_n = int(input('Choose the number of the customer you want to know the activity: '))

    # Check if the input customer number is valid
    if customer_n not in merged_df['customer_id'].values:
        print(f"Invalid customer number: {customer_n}")
        return

    # Retrieve the row for the specified customer
    row = merged_df[merged_df['customer_id'] == customer_n].iloc[0]
    may_rentals = row['may_rental_count']
    june_rentals = row['june_rental_count']

    # Compare rental counts for May and June
    if may_rentals > june_rentals:
        print(f"Customer {customer_n}: More rentals in May")
    elif may_rentals < june_rentals:
        print(f"Customer {customer_n}: More rentals in June")
    else:
        print(f"Customer {customer_n}: Equal rentals in May and June")

input_monthly_activity(may_rent, june_rent)  

Choose the number of the customer you want to know the activity: 222
Customer 222: More rentals in May
